In [3]:
import sys
import os
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

In [4]:
# System alterations

# Set the print options to avoid truncation
np.set_printoptions(threshold=sys.maxsize) # Default threshold is 1000

In [10]:
def load_files(directory):
    """
    Load each .npy file in the specified directory to display all data content

    Parameters:
    - directoryr: str, the path to the directory containing .npy files.
    """
    file_data = {}
    npy_files = [f for f in os.listdir(directory) if f.endswith(".npy")]

    # Get data from all .npy files in directory
    for filename in tqdm(npy_files, desc="Loading .npy files", unit="file"):
        file_path = os.path.join(directory, filename)
        data = np.load(file_path)
        file_data[filename] = data
    
    return file_data

In [7]:
def alter_file_data(directory, coord_index, value=0):
    """
    Modify a specific coordinate (x, y, or z) for all landmarks in every frame
    of each .npy file in the specified directory.

    Parameters:
    - directory: str, the path to the directory containing .npy files.
    - coord_index: int, the index of the coordinate to modify (0 for x, 1 for y, 2 for z).
    - new_value: float, the new value to set for the specified coordinate.
    """
    npy_files = [f for f in os.listdir(directory) if f.endswith(".npy")]
    
    for filename in tqdm(npy_files, desc="Processing .npy files", unit="file"):
        file_path = os.path.join(directory, filename)
        data = np.load(file_path)
        num_frames = data.shape[0]
        num_landmarks = data.shape[1] // 3
        
        # Reshape the data to (num_frames, num_landmarks, 3), alter coord value and save to file
        reshaped_data = data.reshape(num_frames, num_landmarks, 3)
        reshaped_data[:, :, coord_index] = value
        flattened_data = reshaped_data.reshape(num_frames, num_landmarks * 3)
        np.save(file_path, flattened_data)

In [6]:
file_path = os.path.join(os.getcwd(), "RepCount_pose", "test_poses")

In [8]:
# Alter z-coords to 0 after running pre_test.py
alter_file_data(file_path, 2)

Processing .npy files:   0%|          | 0/152 [00:00<?, ?file/s]

In [11]:
# Displays the data showing altered z-coords of a random file
data = load_files(file_path)
key = random.choice(list(data.keys()))
print(f"File {key} (# Frames, # Data Points):", len(data.get(key)), len(data.get(key)[0]))
print(data.get(key))

Loading .npy files:   0%|          | 0/152 [00:00<?, ?file/s]

File stu2_37.npy (# Frames, # Data Points): 1799 99
[[4.91725564e-01 2.37118065e-01 0.00000000e+00 4.69360292e-01
  2.27809787e-01 0.00000000e+00 4.53984082e-01 2.29251370e-01
  0.00000000e+00 4.38023508e-01 2.30927482e-01 0.00000000e+00
  5.14922678e-01 2.27173239e-01 0.00000000e+00 5.29869616e-01
  2.28227124e-01 0.00000000e+00 5.46062291e-01 2.29553774e-01
  0.00000000e+00 4.18202102e-01 2.45306328e-01 0.00000000e+00
  5.75874865e-01 2.45198399e-01 0.00000000e+00 4.69799697e-01
  2.53366828e-01 0.00000000e+00 5.20323336e-01 2.53368407e-01
  0.00000000e+00 3.11885417e-01 3.06355625e-01 0.00000000e+00
  7.00013936e-01 2.97604531e-01 0.00000000e+00 1.67748109e-01
  1.72475204e-01 0.00000000e+00 8.51368427e-01 1.78395733e-01
  0.00000000e+00 5.89482561e-02 2.83772759e-02 0.00000000e+00
  9.58810270e-01 5.25504090e-02 0.00000000e+00 0.00000000e+00
  4.19781450e-03 0.00000000e+00 1.00000000e+00 2.07373705e-02
  0.00000000e+00 5.84404357e-02 0.00000000e+00 0.00000000e+00
  9.58388686e-01 2

In [19]:
def load_csv(csv_file_path):
    df = pd.read_csv(csv_file_path, header=None)
    df = df.drop(columns=[1])
    pose_data_dict = df.set_index(0).T.to_dict('list')
    return pose_data_dict

In [21]:
def alter_csv_data(csv_file_path, coord_index, value=0):
    df = pd.read_csv(csv_file_path, header=None)
    filename = df[0]
    action_type = df[1]
    datapoints = df.iloc[:, 2:]

    coord_indices = range(coord_index, datapoints.shape[1], 3)
    datapoints.iloc[:, coord_indices] = value
    
    updated_df = pd.concat([filename, action_type, datapoints], axis=1)
    updated_df.to_csv(csv_file_path, header=False, index=False)

In [22]:
csv_file_path = os.path.join(os.getcwd(), "RepCount_pose", "annotation_pose", "train.csv")

In [23]:
# Alter z-coords to 0 after running pre_train.py
alter_csv_data(csv_file_path, 2)

In [24]:
# Displays the data showing altered z-coords of a random .csv file entry
pose_data_dict = load_csv(csv_file_path)
for file_path, landmarks in pose_data_dict.items():
    print(f"File: {file_path}")
    print(f"Landmarks: {landmarks}")
    break

File: train/pull_up/salient1/stu5_37.mp4/1.jpg
Landmarks: [315.55087, 120.74478, 0.0, 317.59793, 120.597115, 0.0, 318.86615, 121.041466, 0.0, 320.35458, 121.55323, 0.0, 314.8894, 120.69339, 0.0, 314.12552, 121.091415, 0.0, 313.43884, 121.52831, 0.0, 323.80914, 125.758415, 0.0, 312.34998, 125.9471, 0.0, 317.4511, 124.28728, 0.0, 313.95218, 124.23736, 0.0, 334.92587, 142.10849, 0.0, 295.2929, 143.14166, 0.0, 343.15036, 113.59868, 0.0, 280.36508, 117.206894, 0.0, 346.23145, 77.75645, 0.0, 279.87814, 83.08637, 0.0, 347.6617, 72.584946, 0.0, 276.44717, 76.90932, 0.0, 344.7497, 72.26458, 0.0, 280.0316, 76.48875, 0.0, 343.19516, 74.70981, 0.0, 282.04932, 78.923805, 0.0, 319.08826, 221.32169, 0.0, 297.1822, 218.78343, 0.0, 310.6101, 269.65744, 0.0, 294.94427, 268.13022, 0.0, 303.7521, 319.37097, 0.0, 290.39435, 314.57562, 0.0, 305.12143, 324.70316, 0.0, 294.31647, 319.25433, 0.0, 292.2772, 339.9864, 0.0, 281.58868, 336.76614, 0.0]
